# Visualize Data Issues

This notebook provides a visualization of common data quality issues that may arise within an application, highlighting the importance of robust data validation and cleaning processes.

This notebook is designed to illustrate how data quality issues can manifest in an application. By visualizing these issues, we aim to gain a better understanding of their nature and impact.

## Libraries Import

This cell is responsible for importing the necessary libraries required for our example. Ensuring that all the required libraries are loaded is essential for the analyses that will follow.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

## January Data Analysis

In January, we encountered a timeliness issue with our data, which could also be interpreted as a completeness issue. Let's begin by loading the dataset to investigate this matter further.

- **Timeliness Issue**: This refers to the scenario where data is not available when expected or required.
- **Completeness Issue**: This pertains to instances where data is missing or incomplete.

In [ ]:
df_jan = pd.read_csv("../data/jan/orders.csv",parse_dates=['date'])
df_jan.head()

At first glance, the head of the dataset does not exhibit any peculiarities. Nevertheless, we have received reports of an issue concerning the number of pages visited. To delve deeper into this matter, we'll visualize the average number of pages visited on a daily basis.

In [ ]:
mean_duration = df_jan.groupby('date')['page_visited'].mean()

plt.bar(x=mean_duration.index, height=mean_duration.values)
plt.xticks(rotation=45, ha="right")

plt.xlabel('Date')  
plt.ylabel('Average Page Visits') 

plt.title('Daily Average Page Visits in January')
plt.show()

The visualization reveals a gap in the data, indicating missing averages for certain days. A closer examination of each date confirms this observation, with data absent from the 17th to the 23rd of January.

In [ ]:
df_jan['date'].unique()

The subsequent step involves appending the dataset to the SQL `datalake.orders` table to ensure that our data lake is up-to-date with the latest information.

In [ ]:
con=sqlite3.connect('datalake')
df_jan.to_sql('orders',con,if_exists='append')

## February Data Issues

In February, we encountered a significant issue: writing the dataset to the `datalake.orders` table was infeasible due to a breakdown in the application.

In [ ]:
df_feb = pd.read_csv("../data/feb/orders.csv",parse_dates=['date'])

An inspection of the error reveals a completeness issue, this time attributable to an incomplete schema. Specifically, the field `email_customer` was added. In fact, the field `email` was renamed to `email_customer` in this dataset, which likely caused the problem.

In [ ]:
df_feb.to_sql('orders',con,if_exists='append')

## March Data Anomalies

March brought forth another completeness issue. Due to complications with the cookies provider, there was a noticeable increase in the number of `NaN` values for the `page_visited` column, rising from approximately 5% to 25%.

In [ ]:
df_mar = pd.read_csv("../data/mar/orders.csv",parse_dates=['date'])

In [ ]:
import matplotlib.pyplot as plt

na_jan = sum(df_jan["page_visited"].isna())/df_jan.shape[1]
na_feb = sum(df_feb["page_visited"].isna())/df_feb.shape[1]
na_mar = sum(df_mar["page_visited"].isna())/df_mar.shape[1]

plt.bar(x=['jan', 'feb', 'mar'], height=[na_jan, na_feb, na_mar])


plt.xlabel('Month')  
plt.ylabel('Proportion of Missing Page Visits') 


plt.title('Proportion of Missing Page Visits per Month')

plt.show()

## Summary

Throughout this notebook, we have explored various data issues. Some caused immediate disruptions in the application, while others were more insidious. Recognizing and addressing these issues is crucial for maintaining data integrity and ensuring smooth application functionality.